In [17]:
import pandas as pd

In [18]:
train_moblie_price_file = './dataset/MobilePriceClassification/train.csv'
train_price_data = pd.read_csv(train_moblie_price_file)

In [19]:
train_price_data.columns.values

array(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'], dtype=object)

In [20]:
import torch

In [21]:
# 取得所有標籤
train_y_numpy = train_price_data['price_range'].to_numpy()
train_y = torch.from_numpy(train_y_numpy)

In [22]:
print(train_y.shape)

torch.Size([2000])


In [23]:
'''
{'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi'}
'''

feature_selected = {'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi'}

In [24]:
train_x = torch.from_numpy(train_price_data[feature_selected].to_numpy(dtype='float32'))

C:\Users\Wen2Tee5\AppData\Local\Temp\ipykernel_5620\959956066.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_x = torch.from_numpy(train_price_data[feature_selected].to_numpy(dtype='float32'))


In [25]:
print(f'x shape is {train_x.shape}, x dtype is {train_x.dtype}')

x shape is torch.Size([2000, 20]), x dtype is torch.float32


In [26]:
import torch
from torch import nn


In [27]:
class FCNN(nn.Module):

    def __init__(self) -> None:
        super(FCNN, self).__init__()
        self.linear1 = nn.Linear(20, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 4)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.linear1(x)     
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.output(x)
        return self.softmax(x)

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [29]:
model = FCNN().to(device)

In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [31]:
def train(x, y, model : FCNN, loss_fn, optimizer):

    model.train()
    x, y = x.to(device),y.to(device)
    logits = model(x)
    loss = loss_fn(logits, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss = loss.item()
    
    print(f"[loss: {loss:>7f}]")

def test(x, y, model : FCNN):
    model.eval()
    with torch.no_grad():
        x = x.to(device)
        y = y.to(device)
    
        pred = model(x)
        correct = (pred.argmax(1)==y).sum().item()
    print(correct/400)

In [32]:
for epoch in range(100):
    train(x = train_x,y = train_y, model = model, loss_fn= loss_fn, optimizer=optimizer)

[loss: 1.488161]
[loss: 1.492741]
[loss: 1.492312]
[loss: 1.491339]
[loss: 1.488472]
[loss: 1.477816]
[loss: 1.427871]
[loss: 1.492137]
[loss: 1.393620]
[loss: 1.410907]
[loss: 1.410050]
[loss: 1.430781]
[loss: 1.359318]
[loss: 1.361179]
[loss: 1.370447]
[loss: 1.387304]
[loss: 1.346779]
[loss: 1.345442]
[loss: 1.346767]
[loss: 1.349670]
[loss: 1.347018]
[loss: 1.348902]
[loss: 1.342809]
[loss: 1.346740]
[loss: 1.341788]
[loss: 1.346511]
[loss: 1.339464]
[loss: 1.345073]
[loss: 1.338840]
[loss: 1.344993]
[loss: 1.336969]
[loss: 1.342380]
[loss: 1.338100]
[loss: 1.345133]
[loss: 1.334756]
[loss: 1.336294]
[loss: 1.342194]
[loss: 1.347520]
[loss: 1.334090]
[loss: 1.336248]
[loss: 1.343517]
[loss: 1.333271]
[loss: 1.334074]
[loss: 1.339094]
[loss: 1.347414]
[loss: 1.333470]
[loss: 1.337743]
[loss: 1.346426]
[loss: 1.332934]
[loss: 1.336670]
[loss: 1.345538]
[loss: 1.332413]
[loss: 1.335582]
[loss: 1.344408]
[loss: 1.331765]
[loss: 1.333896]
[loss: 1.341738]
[loss: 1.330921]
[loss: 1.33077